## Classificar as espécies de flores, listadas no dataset iris

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local[*]") \
   .appName("Nome do Projeto") \
   .config("spark.executor.memory", "6gb") \
   .config('spark.sql.debug.maxToStringFields', 2000) \
   .config('spark.debug.maxToStringFields', 2000) \
   .config("spark.sql.caseSensitive", "false") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
# A CSV dataset is pointed to by path.
# The path can be either a single CSV file or a directory of CSV files
source_path = "data/iris.csv"

In [ ]:
# Load in the data
df = spark.read.option("delimiter", ",").option("header", True).csv(source_path)
df.show()

In [ ]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
df.cache()

In [ ]:
df.count()

In [ ]:
df.take(5)

In [ ]:
# Removendo a primeira linha do arquivo (cabeçalho)
df = df.filter(lambda x: "Sepal" not in x)
df.count()

## Limpeza dos Dados

In [ ]:
# Separando as colunas
irisRDD3 = irisRDD2.map(lambda l: l.split(","))

In [ ]:
# Mapeando as colunas
irisRDD4 = irisRDD3.map(lambda p: Row(SEPAL_LENGTH = float(p[0]), SEPAL_WIDTH = float(p[1]), 
                                      PETAL_LENGTH = float(p[2]), PETAL_WIDTH = float(p[3]), 
                                      SPECIES = p[4] ))

In [ ]:
# Criando um Dataframe
irisDF = spark.createDataFrame(irisRDD4)
irisDF.cache()

In [ ]:
irisDF.take(5)

In [ ]:
# Criando um índice numérico para a coluna de label target
stringIndexer = StringIndexer(inputCol = "SPECIES", outputCol = "IDX_SPECIES")
si_model = stringIndexer.fit(irisDF)
irisNormDF = si_model.transform(irisDF)

In [ ]:
irisNormDF.select("SPECIES","IDX_SPECIES").distinct().collect()

## Análise Exploratória de Dados

In [ ]:
# Estatística descritiva
irisNormDF.describe().show()

In [ ]:
# Correlação entre as variáveis
for i in irisNormDF.columns:
    if not( isinstance(irisNormDF.select(i).take(1)[0][0], str)) :
        print( "Correlação da variável Species com", i, irisNormDF.stat.corr('IDX_SPECIES', i))

## Pré-Processamento dos Dados

In [ ]:
# Criando um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row) :
    obj = (row["SPECIES"], row["IDX_SPECIES"], Vectors.dense([row["SEPAL_LENGTH"], row["SEPAL_WIDTH"], 
                                                              row["PETAL_LENGTH"], row["PETAL_WIDTH"]]))
    return obj

In [ ]:
irisRDD5 = irisNormDF.rdd.map(transformaVar)

In [ ]:
irisRDD5.take(5)

In [ ]:
irisDF = spark.createDataFrame(irisRDD5,["species", "label", "features"])
irisDF.select("species","label","features").show(10)
irisDF.cache()

## Machine Learning

In [ ]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = irisDF.randomSplit([0.7, 0.3])

In [ ]:
dados_treino.count()

In [ ]:
dados_teste.count()

In [ ]:
# Construindo o modelo com os dados de treino
dtClassifer = DecisionTreeClassifier(maxDepth = 2, labelCol = "label", featuresCol = "features")
modelo = dtClassifer.fit(dados_treino)

In [ ]:
modelo.numNodes
modelo.depth

In [ ]:
# Previsões com dados de teste
previsoes = modelo.transform(dados_teste)
previsoes.select("prediction","species","label").collect()

In [ ]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "accuracy")
avaliador.evaluate(previsoes)      

In [ ]:
# Resumindo as previsões - Confusion Matrix
previsoes.groupBy("label","prediction").count().show()

In [ ]:
# Stop Spark session
spark.stop()